# ENV

In [1]:
! rm -rf /opt/conda/lib/python3.10/site-packages/aiohttp-3.9.1.dist-info

In [2]:
%pip install -q langchain langchain_community langchain_chroma langchain_experimental langchain-text-splitters
%pip install -q langchain-groq langchain_openai
%pip install -q langchain-huggingface
%pip install -qU langchain-qdrant
%pip install -qU qdrant_client
%pip install -qU langchain-core
%pip install -qU langchain-hub

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.3 requires cubinlinker, which is not installed.
cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.6.0 which is incompatible.
distributed 2024.7.1 requires dask==2024.7.1, but you have dask 2024.9.1 which is incompatible.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.1 which is incompatible.
jupyterlab 4.2.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which 

In [3]:
%pip install transformers torch accelerate --quiet

Note: you may need to restart the kernel to use updated packages.


In [4]:
import getpass
import os
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

#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
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
qdrant_url = os.getenv("QDRANT_URL")
qdrant_api_key = os.getenv("QDRANT_API_KEY")


In [5]:
import torch
# use with kaggle
device_1 = torch.device("cuda:0")
device_2 = torch.device("cuda:1")

# Retriever

In [6]:
# from langchain_huggingface.embeddings import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

In [7]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
import os

text_splitter = SemanticChunker(
    OpenAIEmbeddings()
)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [8]:
from qdrant_client import QdrantClient
from qdrant_client.http import models

# Replace with your Qdrant Cloud URL and API key
qdrant_cloud_url = "https://YOUR_CLOUD_URL"  # Typically "https://cloud.qdrant.io/YOUR_PROJECT_ID"
api_key = "YOUR_API_KEY"

# Create a QdrantClient instance for Qdrant Cloud
qdrant_client = QdrantClient(
    url=qdrant_url,
    api_key=qdrant_api_key
)

# Specify your collection name and query vector
collection_name = "collection_name"

# Perform the search


# Generation

## Function Utils

In [9]:
from sklearn.cluster import KMeans
from typing import Any
from collections import defaultdict
import numpy as np
from qdrant_client.http import models
def multi_perspective_sampling(
    k: int, retrieved_points: list[models.ScoredPoint], seed: int = 1399
) -> list[list[str]]:
    # Generate clusters
    print(f"Finding {k} clusters.")
    algo: Any = KMeans(n_clusters=k, random_state=seed, n_init="auto")
    _vectors = [point.vector for point in retrieved_points]
    clusters: list[int] = algo.fit_predict(X=_vectors)

    # Unique clusters
    unique_clusters: set[int] = set(clusters)

    # Create a dictionary with the members of each cluster
    cluster_dict: defaultdict[int, list[int | None]] = defaultdict(list)
    for index, cluster in enumerate(clusters):
        cluster_dict[cluster].append(index)
    print(f"Clusters distribution: {dict(cluster_dict)}")

    # M subsets
    m: int = min(len(indices) for indices in cluster_dict.values())
    print(f"{m} document subsets will be created.")

    # Generate m unique subsets without replacement
    np.random.seed(seed=seed)
    subsets: list[list[str]] = []

    for _ in range(m):
        subset: list[int] = []
        for cluster in unique_clusters:
            chosen_element: int = np.random.choice(cluster_dict[cluster])
            subset.append(chosen_element)
            cluster_dict[cluster].remove(chosen_element)
        subset_documents = [
            retrieved_points[idx].payload.get("page_content") for idx in subset
        ]
        subsets.append(subset_documents)

    return subsets

In [40]:
def compute_logprobs(outputs, input_ids):
    generated_tokens = outputs.sequences[:, input_ids.shape[-1]:]
    logits_list = outputs.scores

    # Compute log-probabilities for each generated token
    log_probs_list = []
    for i, logits in enumerate(logits_list):
        log_probs = F.log_softmax(logits, dim=-1)  # Convert logits to log-probabilities
        token_log_prob = log_probs[torch.arange(log_probs.size(0)), generated_tokens[:, i]]
        log_probs_list.append(token_log_prob)

    # Stack or concatenate the log-probabilities if needed
    log_probs = torch.stack(log_probs_list, dim=1)  # Shape: (batch_size, max_new_tokens)
    return log_probs

## RAG Drarting

In [19]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import json

# Khởi tạo tokenizer và model
tokenizer_drafter = AutoTokenizer.from_pretrained("ricepaper/vi-gemma-2-2b-function-calling")
model_drafter = AutoModelForCausalLM.from_pretrained(
    "ricepaper/vi-gemma-2-2b-function-calling",
    device_map=device_2,
    torch_dtype=torch.float16,
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
# Draft_RAG tool
tools = """
{
  "name": "draft_rag",
  "description": "Trả lời câu hỏi cùng với lý do vì sao chọn trả lời như vậy.",
  "parameters": {
    "type": "object",
    "properties": {
      "response": {
        "type": "string",
        "description": "Câu trả lời cho câu hỏi dựa trên các thông tin được cung cấp."
      },
      "rationale": {
        "type": "string",
        "description": "Lý do tại sao lại đưa ra trả lời như thế."
      }
    },
    "required": [
      "rationale",
      "response"
    ]
  }
},
"""

In [103]:
# prompt message user
prompt_user: str = f"""Trả lời CÂU HỎI sau đây dựa trên các BẰNG CHỨNG được cung cấp (được phân cách bởi ```). Đồng thời cung cấp lý do cho phản hồi của bạn.
## CÂU HỎI: 
```
{query}
```
## BẰNG CHỨNG: 
```
{evidence}
```
"""
def rag_drafting_generator(query: str, evidence: str):
    # list message
    messages = [
        {"role": "user", "content": f"""Bạn là một người thông hiểu về lịch sử. Nhiệm vụ của bạn là trả lời các câu hỏi trắc nhiệm kèm theo lí do cho câu trả lời của bạn. Chỉ trả lời bằng tiếng Việt. Luôn sử dụng function_call sau {tools}"""},
        {"role": "assistant", "content": "Xin chào, tôi có thể giúp gì cho bạn?"},
        {"role": "user", "content": prompt_user.format(query = query, evidence = evidence)}
    ]
    
    # encode phase
    input_ids = tokenizer_drafter.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)
    
    # generate phase
    outputs = model_drafter.generate(
        input_ids,
        max_new_tokens=300,
        output_scores=True,
        return_dict_in_generate=True,
    )
    response_message = json.loads(tokenizer_drafter.decode(outputs.sequences[0][input_ids.shape[-1]:], skip_special_tokens=True))[0]['arguments']
    
    # compute log_prob
    logprobs = compute_logprobs(outputs, input_ids)
    p_draft = float(torch.exp(torch.mean(logprobs)))
    return response_message, p_draft

## RAG Verifier

In [15]:
!pip install huggingface_hub --quiet

In [16]:
from huggingface_hub import login
login(token=os.getenv("HUGGINGFACE_API_KEY"), add_to_git_credential=True)

Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [17]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer_verifier = AutoTokenizer.from_pretrained('Viet-Mistral/Vistral-7B-Chat')
model_verifier = AutoModelForCausalLM.from_pretrained(
    'Viet-Mistral/Vistral-7B-Chat',
    torch_dtype=torch.bfloat16, # change to torch.float16 if you're using V100
    device_map=device_1,
    use_cache=True,
)

tokenizer_config.json:   0%|          | 0.00/2.52k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/597k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.15M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/169 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

In [98]:
import torch.nn.functional as F
from statistics import mean
def rag_vefifier_generator(query: str, response: str, rationale: str):
    user_prompt = f"""## Câu hỏi: {query}

    ## Câu trả lời: {response}

    ## Lý do: {rationale}

    Lý do có đủ tốt để hỗ trợ cho câu trả lời không? (Có hoặc không)"""
    conversation = [{"role": "system", "content": "Bạn là một trợ lý hữu ích" }]
    conversation.append({"role": "user", "content": user_prompt})    
    input_ids = tokenizer_verifier.apply_chat_template(
        conversation,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model_verifier.device)
    outputs = model_verifier.generate(
            input_ids,
            max_new_tokens=2,
            output_scores=True,
            return_dict_in_generate=True,
            temperature=0.0,
        )
    response_message = tokenizer_verifier.decode(outputs.sequences[0][input_ids.shape[-1]:], skip_special_tokens=True)
    cond: bool = tokenizer_verifier.encode(response_message.lower().strip()) == tokenizer_verifier.encode("có")
    
    logprobs = compute_logprobs(outputs, input_ids)
    p_yes: float = (
        float(torch.exp(torch.mean(logprobs)))
        if cond
        else 0.0
    )
    return (response_message, p_yes)

# Reference

In [ ]:
# end-to-end pipeline
def speculative_rag(
    query: str,
    qdrant_client,
    k: int, 
    seed: int,
    collection_name:str,
    embedding_model,
):
    # compute query embedding
    query_vector = embed_query(query)
    
    # retrieve document
    documents = qdrant_client.search(
        collection_name=collection_name,
        query_vector=query_vector,
        with_vectors=True,
        limit=15
    )
    
    # multi perspective sample
    sampled_docs: list[list[str]] = multi_perspective_sampling(
        k=k, retrieved_points=out, seed=seed
    )
        
    # RAG Drafting
    drafts_answer = []
    for sampled_doc in sampled_docs:
        drafts_answer.append(rag_drafting_generator(query, "\n".join([f"[{idx}] {doc}" for idx, doc in enumerate(sampled_doc, start=1)])))
    
    # RAG Verifier
    rag_verifications = []
    for draft in drafts_answer:
        rag_verifications.append(rag_vefifier_generator)

Testing

In [83]:
query = """Năm 1954, thực dân Pháp có hoạt động nào sau đây?
  A. Xây dựng tập đoàn cứ điểm Điện Biên Phủ.
  B. Rút toàn bộ quân đội ra khỏi miền Bắc Việt Nam.
  C. Kí với triều đình nhà Nguyễn Hiệp ước Giáp Tuất.
  D. Rút toàn bộ quân đội ra khỏi ba nước Đông Dương"""
query_vector = embeddings.embed_query(query)

In [84]:
out = qdrant_client.search(
    collection_name="sgklichsu10",
    query_vector=query_vector,
    with_vectors=True,
    limit=15
)

In [85]:
sampled_docs: list[list[str]] = multi_perspective_sampling(
        k=3, retrieved_points=out, seed=4
    )

Finding 3 clusters.
Clusters distribution: {1: [0, 1, 2, 3, 4, 6, 7, 9, 11, 12, 14], 2: [5, 8], 0: [10, 13]}
2 document subsets will be created.


In [89]:
(res_message, p_draft) = rag_drafting_generator(query, "\n".join([f"[{idx}] {doc}" for idx, doc in enumerate(sampled_docs[0], start=1)]))

In [102]:
json.loads(res_message)

[{'name': 'draft_rag',
  'arguments': {'response': 'Năm 1954, thực dân Pháp có xây dựng tập đoàn cứ điểm Điện Biên Phủ',
   'rationale': 'Căn cứ Điện Biên Phủ là một trong những điểm tập trung quân sự quan trọng nhất của thực dân Pháp trong Chiến tranh Việt Nam.'}}]

In [99]:
response_message, p_yes = rag_vefifier_generator(query, json.loads(res_message)[0]['arguments']['response'], json.loads(res_message)[0]['arguments']['rationale'])

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [40]:
list_test_draft = []
for sampled_doc in sampled_docs:
    test_draft = rag_drafting_generator(query = query, evidence="\n".join([f"[{idx}] {doc}" for idx, doc in enumerate(sampled_doc, start=1)]),)
    list_test_draft.append(test_draft)

In [41]:
list_test_draft

['[{"name": "draft_rag", "arguments": {"response": "Chính phủ lâm thời tự san định mang tên Chính phủ Vệ quốc. Khí quản Phổ tiến về Pa-ri và bao vây thành phố, “Chính phủ Vệ quốc” đã trở thành “Chính phủ tiến vương nước Phổ”. Nhưng nhân dân Pa-ri đã tổ chức kháng cự các đòn vây Quốc dân quân, bảo vệ thành tuyên bố vệ thủ đô. Trưa 18-3, theo lệnh của Ủy ban trung ương Quốc dân quản, các tiểu đoàn tiến vào trung tâm thủ đô, chiếm được các cơ quan chính phủ, nhà ga, sở cảnh sát và tòa Thị chính.", "rationale": "Chính phủ lâm thời tự san định mang tên Chính phủ Vệ quốc. Khí quản Phổ tiến về Pa-ri và bao vây thành phố, “Chính phủ Vệ quốc” đã trở thành “Chính phủ tiến vương nước Phổ”. Nhưng nhân dân Pa-ri đã tổ chức kháng cự các đòn vây Quốc dân quân, bảo vệ thành tuyên bố vệ thủ đô. Trưa 18-3, theo lệnh của Ủy ban trung ương Quốc dân quản, các tiểu đoàn tiến vào trung tâm thủ đô, chiếm được các cơ quan chính phủ, nhà ga, sở cảnh sát và tòa Thị chính."}]',
 '{\n  "name": "draft_rag",\n  "arg

In [45]:
test_res = rag_drafting_generator(query = query, evidence="\n".join([f"[{idx}] {doc}" for idx, doc in enumerate(sampled_docs[1], start=1)]),)